## Dependency

In [1]:
import yaml #pip install pyYAML
from utils.utils import convert_params
from sklearn.metrics import classification_report
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
import pickle
import os

## path of data

In [ ]:

data_path =  '..\Bootcamp_Machine_Learning_Production_Data_Preparation\dataset'

def load_pickle(child_path): #load data.pickle 
    data = pickle.load(open(os.path.join(data_path, child_path), 'rb'))
    return data